### Import the relevant libraries

In [3]:
from sklearn import preprocessing
import numpy as np
import tensorflow as tf

###### Extract data from csv

In [4]:
raw_csv_data = np.loadtxt('glass.csv', delimiter=',')
unscaled_inputs_all=raw_csv_data[:,:-1]
targets_all=raw_csv_data[:,-1]

###### Standardise

In [5]:
scaled_inputs = preprocessing.scale(unscaled_inputs_all)

###### Shuffle

In [6]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs=scaled_inputs[shuffled_indices]
shuffled_targets = targets_all[shuffled_indices]

###### Split (Train, validation, test)

In [7]:
samples_count=shuffled_inputs.shape[0]
train_samples_count= int(0.8*samples_count)
validation_samples_count = int(0.1* samples_count)
test_samples_count=samples_count-train_samples_count-validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+ validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+ validation_samples_count:]

###### Save 3 datasets in npz

In [8]:
np.savez('Glass_data_train',inputs=train_inputs,targets=train_targets)
np.savez('Glass_data_validation',inputs=validation_inputs,targets=validation_targets)
np.savez('Glass_data_test',inputs=test_inputs,targets=test_targets)

### Data

In [9]:
npz = np.load('Glass_data_train.npz')
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)
npz = np.load('Glass_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
npz = np.load('Glass_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

### Model
Outline, optimizers, loss, early stopping and training

In [22]:
input_size = 9
output_size = 8
hidden_layer_size = 50
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 3rd hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


###The optimizer and the loss function
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
batch_size = 25
max_epochs = 100

# Early stopping mechanism
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs, 
          train_targets, 
          batch_size=batch_size, 
          epochs=max_epochs, 
            callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets), 
          verbose = 2
          )  

Epoch 1/100
7/7 - 0s - loss: 2.1295 - accuracy: 0.0994 - val_loss: 1.9996 - val_accuracy: 0.1429
Epoch 2/100
7/7 - 0s - loss: 1.9666 - accuracy: 0.2515 - val_loss: 1.8569 - val_accuracy: 0.5238
Epoch 3/100
7/7 - 0s - loss: 1.8295 - accuracy: 0.4386 - val_loss: 1.7252 - val_accuracy: 0.5238
Epoch 4/100
7/7 - 0s - loss: 1.6941 - accuracy: 0.4620 - val_loss: 1.6015 - val_accuracy: 0.5238
Epoch 5/100
7/7 - 0s - loss: 1.5541 - accuracy: 0.4561 - val_loss: 1.4887 - val_accuracy: 0.5238
Epoch 6/100
7/7 - 0s - loss: 1.4184 - accuracy: 0.4795 - val_loss: 1.3807 - val_accuracy: 0.5238
Epoch 7/100
7/7 - 0s - loss: 1.2845 - accuracy: 0.5205 - val_loss: 1.2847 - val_accuracy: 0.5714
Epoch 8/100
7/7 - 0s - loss: 1.1631 - accuracy: 0.6257 - val_loss: 1.2128 - val_accuracy: 0.6190
Epoch 9/100
7/7 - 0s - loss: 1.0719 - accuracy: 0.6491 - val_loss: 1.1582 - val_accuracy: 0.5238
Epoch 10/100
7/7 - 0s - loss: 0.9983 - accuracy: 0.6550 - val_loss: 1.1168 - val_accuracy: 0.5714
Epoch 11/100
7/7 - 0s - loss:

# Testing the model on a completely new dataset

In [23]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

1/1 [==============================] - 0s 18ms/step - loss: 0.6370 - accuracy: 0.8636


In [24]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.64. Test accuracy: 86.36%


Here, we conclude that out model was able to predict at an accuracy of 86.36%